In [129]:
class Character:
    def __init__(self, name, hp, level, position=(0, 0), inventory=None):
        self.name = name
        self.hp = hp
        self.level = level
        self.position = position
        self.inventory = inventory if inventory else Inventory(self.MAX_INVENTORY_SPACE)

    def move(self, direction):
        x, y = self.position
        if direction == 'u':
            y -= 1
        elif direction == 'd':
            y += 1
        elif direction == 'l':
            x -= 1
        elif direction == 'r':
            x += 1
        else:
            print("Invalid direction.")
            return
        self.position = (x, y)
        print(f"{self.name} moves to position ({x}, {y})!")

    def print_info(self):
        print(f"Name: {self.name}")
        print(f"HP: {self.hp}")
        print(f"Level: {self.level}")
        print(f"Position: {self.position}")


class Player(Character):
    MAX_INVENTORY_SPACE = 50  # Player's maximum inventory space
    EXP_TO_LEVEL_UP = 10  # Amount of experience points needed to level up
    EXTRA_HP_ON_LEVEL_UP = 10  # Extra HP gained on level up

    def __init__(self, name, hp, level, position=(0, 0), inventory=None):
        super().__init__(name, hp, level, position)
        self.inventory_space = self.MAX_INVENTORY_SPACE
        if inventory:
            self.inventory = inventory
        self.exp = 0

    def add_exp(self, amount):
        self.exp += amount
        print(f"{self.name} gained {amount} experience points!")
        if self.exp >= self.EXP_TO_LEVEL_UP:
            self.level_up()

    def level_up(self):
        self.level += 1
        self.hp += self.EXTRA_HP_ON_LEVEL_UP
        print(f"{self.name} leveled up to level {self.level}!")
        print(f"{self.name} gained {self.EXTRA_HP_ON_LEVEL_UP} extra HP and fully restored health!")
        self.hp = self.max_hp  # Restore player's HP to maximum
        self.exp -= self.EXP_TO_LEVEL_UP  # Deduct the experience points used to level up

    @property
    def max_hp(self):
        return 50 + (self.level - 1) * 10  # Assuming base HP is 50 and each level adds 10 HP

    def add_item_to_inventory(self, item):
        added = self.inventory.add_item(item)
        if added:
            print(f"{item} added to {self.name}'s inventory.")
        else:
            print("Failed to add item to inventory.")

    def view_inventory(self):
        print(f"Inventory of {self.name}:")
        self.inventory.print_item_names_in_inventory('ItemData.json')
        
    def save_inventory(self):
            self.inventory.save_inventory(self.name)



class Enemy(Character):
    MAX_INVENTORY_SPACE = 5  # Enemy's maximum inventory space

    def __init__(self, name, hp, level, position=(0, 0), inventory=None):
        super().__init__(name, hp, level, position)
        self.inventory_space = self.MAX_INVENTORY_SPACE
        if inventory:
            self.inventory = inventory

    def add_item_to_inventory(self, item):
        added = self.inventory.add_item(item)
        if added:
            print(f"{item} added to {self.name}'s inventory.")
        else:
            print("Failed to add item to inventory.")
            
    def save_inventory(self):
            self.inventory.save_inventory(self.name)

class Inventory:
    def __init__(self, max_inventory_space):
        self.max_inventory_space = max_inventory_space
        self.items = {
            'Weapons': {"Category": 'Weapons', "Amount": 0, "Ids": [], "Stack_Size": 1},
            'Food': {"Category": 'Food', "Amount": 0, "Ids": [], "Stack_Size": 30},
            'General': {"Category": 'General', "Amount": 0, "Ids": [], "Stack_Size": 9999},
            'Healing': {"Category": 'Healing', "Amount": 0, "Ids": [], "Stack_Size": 16},
            'Armour': {"Category": 'Armour', "Amount": 0, "Ids": [], "Stack_Size": 7},
            'Magic': {"Category": 'Magic', "Amount": 0, "Ids": [], "Stack_Size": 32}
        }

    def add_item(self, item):
        if isinstance(item, str):  
            item_data = None
            with open('ItemData.json', 'r') as file:
                data = json.load(file)["items"]
                for data_item in data:
                    if 'name' in data_item and data_item['name'].lower() == item.lower():
                        item_data = data_item
                        break
            
            if item_data is None:
                print(f"Item '{item}' not found in ItemData.")
                return False
            else:
                item = item_data
    
        item_type = item.get('category')
        item_id = item.get('id')
    
        if item_type in self.items:
            # Check if the item already exists in the inventory
            for idx, existing_id in enumerate(self.items[item_type]["Ids"]):
                if existing_id == item_id:
                    # If item exists, increase the amount and return
                    if self.items[item_type]["Amount"] < self.items[item_type]["Stack_Size"]:
                        self.items[item_type]["Amount"] += 1
                        return True
                    else:
                        print("Inventory is full.")
                        return False
            # If item doesn't exist, add a new entry
            if self.items[item_type]["Amount"] < self.items[item_type]["Stack_Size"]:
                self.items[item_type]["Amount"] += 1
                self.items[item_type]["Ids"].append(item_id)
                # Add the category field to the item
                item["category"] = item_type
                return True
            else:
                print("Inventory is full.")
                return False
        else:
            print("Item cannot be added to inventory.")
            return False

    def remove_item(self, item_type):
        if item_type in self.items and self.items[item_type]["Amount"] > 0:
            removed_id = self.items[item_type]["Ids"].pop()
            self.items[item_type]["Amount"] -= 1
            print(f"Removed {item_type} with ID {removed_id}")
        else:
            print("No such item in inventory.")

    def get_total_items(self):
        total = 0
        for item in self.items.values():
            total += item["Amount"]
        return total

    def save_inventory(self, player_name):
        filename = f"{player_name}_inventory.json"
        with open(filename, 'w') as results_file:
            for item in self.items.values():
                results_file.write(json.dumps(item) + '\n')

    def print_item_names_in_inventory(self, item_data_file):
        item_names = {}
        with open(item_data_file, 'r') as file:
            data = json.load(file)["items"]
            for item in data:
                if 'id' in item and 'name' in item:
                    item_names[item['id']] = item['name']
                else:
                    print("Item doesn't have a name or id field, skipping.")
        
        for item_type, item_info in self.items.items():
            for item_id in item_info["Ids"]:
                if item_id in item_names:
                    print(f"Item in inventory: {item_names[item_id]}")


class Map:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[' ' for _ in range(width)] for _ in range(height)]
        self.enemies = []

    def add_enemy(self, enemy):
        self.enemies.append(enemy)

    def print_map(self, characters):
        for y in range(self.height):
            print('\n' + '+---' * self.width + '+')
            for x in range(self.width):
                symbol = ' '
                for character in characters:
                    if character.position == (x, y):
                        if symbol == ' ':
                            symbol = 'P' if isinstance(character, Player) else 'E'  # P for player, E for enemy
                        else:
                            symbol = 'X'  # Indicates more than one character in the same position
                print(f'| {symbol} ', end='')
            print('|', end='')
        print('\n' + '+---' * self.width + '+')
        
    def check_battle(self, player):
        for enemy in self.enemies:
            if player.position == enemy.position:
                return True
        return False

class Battle:
    def __init__(self, player, enemy):
        self.player = player
        self.enemy = enemy

    def roll_dice(self):
        return random.randint(1, 20)

    def start_battle(self):
        print("Battle started!")
    
        while self.player.hp > 0 and self.enemy.hp > 0:
            # Player's turn
            input(f"{self.player.name}, press Enter to roll the dice for your attack...")
            player_roll = self.roll_dice()
            print(f"{self.player.name} rolls: {player_roll}")
            self.attack(self.player, self.enemy, player_roll)
    
            if self.enemy.hp <= 0:
                print(f"{self.enemy.name} has been defeated!")
                break  # Exit the battle loop if the enemy is defeated
    
            # Enemy's turn
            enemy_roll = self.roll_dice()
            print(f"{self.enemy.name} rolls: {enemy_roll}")
            self.attack(self.enemy, self.player, enemy_roll)
    
            if self.player.hp <= 0:
                print(f"{self.player.name} has been defeated!")
                break  # Exit the battle loop if the player is defeated
    
            # Print enemy's HP after each attack
            print(f"{self.enemy.name}'s HP: {self.enemy.hp}")
            
        # After the battle loop ends
        print("Returning to the map.")

    def attack(self, attacker, defender, roll):
        print(f"{attacker.name} attacks {defender.name}!")
        if roll >= 10:  # If roll is 10 or more, the attack hits
            damage = self.get_attack_damage(attacker) + random.randint(1, 6)  # Roll a 1d6 dice and add it to damage
            print(f"{attacker.name} deals {damage} damage to {defender.name}!")
            defender.hp -= damage
            if defender.hp <= 0:
                print(f"{defender.name} has been defeated!")
                if isinstance(defender, Enemy):
                    self.player.add_exp(10)  # Award 10 XP to the player if the enemy is defeated
        else:
            print(f"{attacker.name} missed the attack!")


    def get_attack_damage(self, character):
        weapon_ids = character.inventory.items["Weapons"]["Ids"]
        damage = 0
        # Iterate over the weapon IDs to find the damage
        for item_id in weapon_ids:
            try:
                # Get the corresponding item dictionary
                item = character.inventory.items["Weapons"][item_id]
                # Check if the item has a damage field
                if "damage" in item:
                    # Get the damage value from the ItemData.json file
                    damage += self.get_item_damage(item_id)
            except KeyError:
                print(f"Item with ID {item_id} not found in inventory.")
        # If character has no weapon or damage data, return a default value
        return damage if damage else 1
        
    def get_item_damage(self, item_id):
        with open('ItemData.json', 'r') as file:
            data = json.load(file)["items"]
            for item in data:
                if 'id' in item and item['id'] == item_id:
                    return item.get('damage', 0)
        return 0  # Return 0 if item not found or has no damage field


In [131]:
map_width = 10
map_height = 10
game_map = Map(map_width, map_height)

# Create characters
player = Player("Ant", hp=50, level=1)
game_map.add_enemy(Enemy("Goblin", hp=20, level=1, position=(2, 2)))
game_map.add_enemy(Enemy("Orc", hp=30, level=1, position=(5, 5)))
game_map.add_enemy(Enemy("Spider", hp=10, level=1, position=(4, 2)))

enemy.add_item_to_inventory("wooden sword") 
player.add_item_to_inventory("wooden sword") 
player.save_inventory()
player.view_inventory()
player.print_info()

# Print the initial map
game_map.print_map([player] + game_map.enemies)

# Main game loop
while True:
    direction = input("Enter direction (l for left, r for right, u for up, d for down, q to quit): ").lower()
    
    if direction == 'q':
        print("Exiting game.")
        break
    
    if direction in ['l', 'r', 'u', 'd']:
        player.move(direction)
        game_map.print_map([player] + game_map.enemies)
        
        # Check for battle
        if game_map.check_battle(player):
            enemy = [enemy for enemy in game_map.enemies if enemy.position == player.position][0]
            battle = Battle(player, enemy)
            battle.start_battle()
            
            # Remove defeated enemy from the map
            game_map.enemies.remove(enemy)
            game_map.print_map([player] + game_map.enemies)  # Print updated map after removing enemy
    else:
        print("Invalid direction. Please enter l, r, u, d, or q to quit.")


wooden sword added to Goblin's inventory.
wooden sword added to Ant's inventory.
Inventory of Ant:
Item in inventory: Wooden Sword
Name: Ant
HP: 50
Level: 1
Position: (0, 0)

+---+---+---+---+---+---+---+---+---+---+
| P |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   | E |   | E |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   | E |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |  

Enter direction (l for left, r for right, u for up, d for down, q to quit):  r


Ant moves to position (1, 0)!

+---+---+---+---+---+---+---+---+---+---+
|   | P |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   | E |   | E |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   | E |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+


Enter direction (l for left, r for right, u for up, d for down, q to quit):  r


Ant moves to position (2, 0)!

+---+---+---+---+---+---+---+---+---+---+
|   |   | P |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   | E |   | E |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   | E |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+


...

Ant, press Enter to roll the dice for your attack... 


Ant rolls: 13
Ant attacks Orc!
Item with ID 127 not found in inventory.
Ant deals 6 damage to Orc!
Orc rolls: 8
Orc attacks Ant!
Orc missed the attack!
Orc's HP: 4


Ant, press Enter to roll the dice for your attack... 


Ant rolls: 13
Ant attacks Orc!
Item with ID 127 not found in inventory.
Ant deals 7 damage to Orc!
Orc has been defeated!
Ant gained 10 experience points!
Ant leveled up to level 4!
Ant gained 10 extra HP and fully restored health!
Orc has been defeated!
Returning to the map.

+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   | P |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+---+
|   |   

KeyboardInterrupt: Interrupted by user

In [19]:
'''import json

class Inventory:
    def __init__(self, max_inventory_space):
        self.max_inventory_space = max_inventory_space
        self.items = {
            'Weapons': {"Category": 'Weapons', "Amount": 0, "Ids": [], "Stack_Size": 1},
            'Food': {"Category": 'Food', "Amount": 0, "Ids": [], "Stack_Size": 30},
            'General': {"Category": 'General', "Amount": 0, "Ids": [], "Stack_Size": 9999},
            'Healing': {"Category": 'Healing', "Amount": 0, "Ids": [], "Stack_Size": 16},
            'Armour': {"Category": 'Armour', "Amount": 0, "Ids": [], "Stack_Size": 7},
            'Magic': {"Category": 'Magic', "Amount": 0, "Ids": [], "Stack_Size": 32}
        }

    def add_item(self, item):
        if isinstance(item, str):  
            item_data = None
            with open('ItemData.json', 'r') as file:
                data = json.load(file)["items"]
                for data_item in data:
                    if 'name' in data_item and data_item['name'].lower() == item.lower():
                        item_data = data_item
                        break
            
            if item_data is None:
                print(f"Item '{item}' not found in ItemData.")
                return False
            else:
                item = item_data
    
        item_type = item.get('category')
        item_id = item.get('id')
    
        if item_type in self.items:
            # Check if the item already exists in the inventory
            for idx, existing_id in enumerate(self.items[item_type]["Ids"]):
                if existing_id == item_id:
                    # If item exists, increase the amount and return
                    if self.items[item_type]["Amount"] < self.items[item_type]["Stack_Size"]:
                        self.items[item_type]["Amount"] += 1
                        return True
                    else:
                        print("Inventory is full.")
                        return False
            # If item doesn't exist, add a new entry
            if self.items[item_type]["Amount"] < self.items[item_type]["Stack_Size"]:
                self.items[item_type]["Amount"] += 1
                self.items[item_type]["Ids"].append(item_id)
                return True
            else:
                print("Inventory is full.")
                return False
        else:
            print("Item cannot be added to inventory.")
            return False

    def remove_item(self, item_type):
        if item_type in self.items and self.items[item_type]["Amount"] > 0:
            removed_id = self.items[item_type]["Ids"].pop()
            self.items[item_type]["Amount"] -= 1
            print(f"Removed {item_type} with ID {removed_id}")
        else:
            print("No such item in inventory.")
c
    def get_total_items(self):
        total = 0
        for item in self.items.values():
            total += item["Amount"]
        return total

    def save_inventory(self, filename):
        with open(filename, 'w') as results_file:
            for item in self.items.values():
                results_file.write(json.dumps(item) + '\n')

    def print_item_names_in_inventory(self, item_data_file):
        item_names = {}
        with open(item_data_file, 'r') as file:
            data = json.load(file)["items"]
            for item in data:
                if 'id' in item and 'name' in item:
                    item_names[item['id']] = item['name']
                else:
                    print("Item doesn't have a name or id field, skipping.")
        
        for item_type, item_info in self.items.items():
            for item_id in item_info["Ids"]:
                if item_id in item_names:
                    print(f"Item in inventory: {item_names[item_id]}")


class Character:
    def __init__(self, name, hp, level, char_class, spell_slots=None):
        self.name = name
        self.hp = hp
        self.level = level
        self.char_class = char_class
        self.inventory = Inventory(max_inventory_space=20) 
        self.spell_slots = spell_slots if spell_slots else 0

    def attack(self):
        print(f"{self.name} attacks!")

    def cast_spell(self):
        if self.char_class == "Mage":
            if self.spell_slots > 0:
                print(f"{self.name} casts a spell!")
                self.spell_slots -= 1
            else:
                print(f"{self.name} is out of spell slots!")
        else:
            print(f"{self.name} can't cast spells!")

    def add_item(self, item):
        added = self.inventory.add_item(item)
        if added:
            print(f"{item['name']} added to {self.name}'s inventory.")
        else:
            print(f"Failed to add {item['name']} to {self.name}'s inventory.")

    def remove_item(self, item_name):
        self.inventory.remove_item(item_name)

    def display_inventory(self):
        self.inventory.print_item_names_in_inventory('ItemData.json')

    def get_item_details(self, item_name):
        self.inventory.print_item_names_in_inventory('ItemData.json')
        self.inventory.get_item_details(item_name)

    def equip_item(self):
        self.inventory.print_item_names_in_inventory('ItemData.json')
        self.inventory.equip_item()

    def unequip_item(self):
        self.inventory.print_item_names_in_inventory('ItemData.json')
        self.inventory.unequip_item()


class Player(Character):
    def __init__(self, name, hp, level, char_class, spell_slots=None):
        super().__init__(name, hp, level, char_class, spell_slots)

    def level_up(self):
        self.level += 1
        print(f"{self.name} leveled up to level {self.level}!")

    def rest(self):
        self.hp = 100
        print(f"{self.name} rested and recovered to full health.")'''


IndentationError: unexpected indent (2065064183.py, line 65)

In [2]:
'''import json

class Character:
    def __init__(self, name, hp, level, char_class, spell_slots=None):
        self.name = name
        self.hp = hp
        self.level = level
        self.char_class = char_class
        self.inventory = Inventory(max_inventory_space=20)  # Example: 20 slots for the character's inventory
        self.spell_slots = spell_slots if spell_slots else 0

    def attack(self):
        print(f"{self.name} attacks!")

    def cast_spell(self):
        if self.char_class == "Mage":
            if self.spell_slots > 0:
                print(f"{self.name} casts a spell!")
                self.spell_slots -= 1
            else:
                print(f"{self.name} is out of spell slots!")
        else:
            print(f"{self.name} can't cast spells!")

    def add_item(self, item):
        added = self.inventory.add_item(item)
        if added:
            print(f"{item['name']} added to {self.name}'s inventory.")
        else:
            print(f"Failed to add {item['name']} to {self.name}'s inventory.")

    def remove_item(self, item_name):
        self.inventory.remove_item(item_name)

    def display_inventory(self):
        self.inventory.print_item_names_in_inventory('ItemData.json')

    def get_item_details(self, item_name):
        self.inventory.print_item_names_in_inventory('ItemData.json')
        self.inventory.get_item_details(item_name)

    def equip_item(self):
        self.inventory.print_item_names_in_inventory('ItemData.json')
        self.inventory.equip_item()

    def unequip_item(self):
        self.inventory.print_item_names_in_inventory('ItemData.json')
        self.inventory.unequip_item()


class Player(Character):
    def __init__(self, name, hp, level, char_class, spell_slots=None):
        super().__init__(name, hp, level, char_class, spell_slots)

    def level_up(self):
        self.level += 1
        print(f"{self.name} leveled up to level {self.level}!")

    def rest(self):
        self.hp = 100
        print(f"{self.name} rested and recovered to full health.")'''


bill's Inventory:
Staff - A gnarled wooden staff.
Potion - A small vial of glowing blue liquid.


In [ ]:
'''class Skill_Tree(Character):
    def __init__(self, name, hp, level, char_class, skill_points):
        super().__init__(name, hp, level, char_class)
        self.skill_points = skill_points
    
    def select_class(self, char_class):
        self.char_class = char_class
        print(f"{self.name} selects {char_class} class.")

    def show_skill_tree(self):
        print(f"{self.char_class} Skill Tree:")
        # Implement logic to show the skill tree based on the character's class
        if self.char_class == "Warrior":
            print("1. Sword Mastery")
            print("2. Shield Bash")
            print("3. Warcry")
        elif self.char_class == "Mage":
            print("1. Fireball")
            print("2. Ice Blast")
            print("3. Teleport")
        elif self.char_class == "Rogue":
            print("1. Stealth")
            print("2. Backstab")
            print("3. Poison Strike")
        else:
            print("No skill tree available for this class.")
     def learn_new_skill(self, skill_name):
        if self.skill_points > 0:
            self.skills.append(skill_name)
            self.skill_points -= 1
            print(f"{self.name} learns {skill_name}!")
        else:
            print(f"{self.name} doesn't have enough skill points to learn {skill_name}.")

     def reset(self):
        self.hp = 100
        self.level = 1
        self.skill_points = 0
        self.skills = []
        print(f"{self.name}'s attributes have been reset.")

'''

In [ ]:
#List of Classes
'''class Warrior(Character):
    def __init__(self, name, hp, level):
        super().__init__(name, hp, level, "Warrior")

    def attack(self):
        print(f"{self.name} swings a mighty sword!")


class Mage(Character):
    def __init__(self, name, hp, level, spell_slots):
        super().__init__(name, hp, level, "Mage", spell_slots)

    def attack(self):
        print(f"{self.name} launches a magic missile!")


class Rogue(Character):
    def __init__(self, name, hp, level):
        super().__init__(name, hp, level, "Rogue")

    def attack(self):
        print(f"{self.name} strikes from the shadows with a dagger!")
        
#Diffrent characters name and stats
        warrior = Warrior("Karlach", 100, 1) # (Hp,Level)
        mage = Mage("Gale", 80, 12, 1)
        rogue = Rogue("Astarion", 90, 1)

     '''

In [32]:
itemdata = 'ItemData.json'
max_inventory_space = 50
inventory = Inventory(max_inventory_space)

with open(itemdata, 'r') as file:
    data = json.load(file)["items"]

# Add one item of each category
'''for item in data:
    if 'name' in item:
        print(f"Added {item['name']} to inventory.")
        inventory.add_item(item)
    else:
        print("Item doesn't have a name field, skipping.")'''



for _ in range(3):
         inventory.add_item('aIr GeM')
         inventory.add_item("Alchemist's Scroll")
            


inventory.save_inventory('Inventory.txt')


'''# Print names of items in inventory
inventory.print_item_names_in_inventory('ItemData.json')'''

Item in inventory: Air Gem
Item in inventory: Alchemist's Scroll


In [13]:
import json

def add_damage_category(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    for item in data['items']:
            item['Value'] = ""
    
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

# Replace 'ItemData - Copy.json' with the actual file path
add_damage_category('ItemData - Copy.json')
